## Day 10: Knot Hash

http://adventofcode.com/2017/day/10

### Part 1

Circular lists again, it's probably time to define some utility functions. And it's probably time to stop using mutable data structures (unless they're really convenient).

In [1]:
from itertools import cycle, islice
from pyrsistent import pvector 

def cget(v, i):
    return v[i % len(v)]

def cset(v, i, val):
    return v.set(i % len(v), val)

def cslice(v, start, stop):
    return islice(cycle(v), start, stop)

In [2]:
def knot_step(string, length, current):
    r = reversed(pvector(cslice(string, current, current + length)))
    result = pvector(string)
    for i, c in zip(range(current, current + length), r):
        result = cset(result, i, c)
    return result

def knot(string, lengths):
    current = 0
    skip = 0
    
    for length in lengths:
        string = knot_step(string, length, current)
        current = current + length + skip
        skip += 1
        
    return string

In [3]:
test_lengths = [3, 4, 1, 5, 0]
test_string = pvector(range(5))

knot(test_string, test_lengths)

pvector([3, 4, 2, 1, 0])

Jolly good.

In [4]:
input_lengths = (int(x) for x in open('input', 'r').read().split(','))
input_string = pvector(range(256))

x = knot(input_string, input_lengths)
x[0] * x[1]

15990

### Part 2

Good grief. I'll come back to this one.